In [1]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Lab_Analyses.Spine_Analysis_v2 import spine_utilities as s_utils

sns.set()
sns.set_style("ticks")

In [2]:
## Specify data to load
mouse_id = "JW202"
days = ["Early"]
fov_type = "apical"

# Load the data
full_data = s_utils.load_spine_datasets(mouse_id, days, fov_type)

In [4]:
# Grab relevant data
data = full_data["FOV3_apical"]["Early"]

In [5]:
data.__dict__.keys()

dict_keys(['mouse_id', 'session', 'date', 'imaging_parameters', 'time', 'lever_force_resample', 'lever_force_smooth', 'lever_velocity_envelope', 'lever_active', 'rewarded_movement_force', 'rewarded_movement_binary', 'binary_cue', 'reward_delivery', 'punish_delivery', 'spine_flags', 'spine_groupings', 'spine_positions', 'spine_volume', 'corrected_spine_volume', 'spine_GluSnFr_dFoF', 'spine_GluSnFr_processed_dFoF', 'spine_GluSnFr_activity', 'spine_GluSnFr_floored', 'spine_calcium_dFoF', 'spine_calcium_processed_dFoF', 'spine_calcium_activity', 'spine_calcium_floored', 'movement_spines', 'reward_movement_spines', 'silent_spines', 'reward_silent_spines', 'dendrite_length', 'dendrite_calcium_dFoF', 'dendrite_calcium_processed_dFoF', 'dendrite_calcium_activity', 'dendrite_calcium_floored', 'movement_dendrites', 'reward_movement_dendrites', 'silent_dendrites', 'reward_silent_dendrites', 'poly_dendrite_positions', 'poly_dendrite_calcium_dFoF', 'poly_dendrite_calcium_processed_dFoF', 'followup_

In [6]:
data.imaging_parameters

{'Imaging Sensor': 'iGluSnFr3',
 'Zoom': 8.0,
 'Sampling Rate': 60.0,
 'Calculate dFoF': True,
 'Deconvolve': False,
 'Calculate Volume': True,
 'Correct Bout Separation': True,
 'Ds Ratio': 20,
 'Smooth Window': 0.5,
 'Threshold': 2.0,
 'Artifact Frames': [(6300, 6600),
  (20800, 21000),
  (22500, 23200),
  (30500, 30800)],
 'Spine Groupings': [],
 'Threshold Values': {'Dendrite': {'Upper Threshold': [0.2],
   'Lower Threshold': [0.040043659101153754],
   'Artifact Limit': [0.08352171876751074]},
  'Dendrite Poly': [{'Upper Threshold': [0.2790024301799002,
     0.2821977805482691,
     0.3039511474594695,
     0.28787688342325735,
     0.281825286595847,
     0.27538417475867244,
     0.2775907065759167,
     0.2827636740731594,
     0.29182280774172586,
     0.30111480205510466,
     0.2329054603190103,
     0.2517941531021508,
     0.27041726267566546,
     0.263103099669681,
     0.2974130852605573,
     0.2593331073174318,
     0.28172453583385254,
     0.2783133168769923,
     0.

In [7]:
# Get activity related data
spine_activity = data.spine_GluSnFr_activity
spine_dFoF = data.spine_GluSnFr_processed_dFoF
dendrite_activity = data.dendrite_calcium_activity
dendrite_dFoF = data.dendrite_calcium_processed_dFoF

In [ ]:
from Lab_Analyses.Utilities.load_mat_files import load_mat
matlab_path = r"Z:\People\Nathan\Elim Spine Manuscript\GluSnFR2_data"
matlat_data = load_mat("NH085_Aligned", path=matlab_path)

In [ ]:
#spine_dFoF = matlat_data[-1].ProcessedSpineActivity.T
#spine_activity = matlat_data[-1].BinarizedOverallSpineData.T
import sklearn as sk

#a = np.array([[1,2],
#              [10,10],
#              [5, 15]])
a = np.array([1,2,5,9,1,2,6])
scaler = sk.preprocessing.MinMaxScaler()
b = scaler.fit_transform(a)
b

In [8]:
# Mask active and inactive portions
def get_active_inactive(dFoF, active):
    inactive_mask = (active == 1)
    active_mask = (active == 0)
    active_dFoF = np.copy(dFoF)
    inactive_dFoF = np.copy(dFoF)
    active_dFoF[active_mask] = np.nan
    inactive_dFoF[inactive_mask] = np.nan
    
    return active_dFoF, inactive_dFoF

In [9]:
spine_active = np.zeros(spine_activity.shape) * np.nan
spine_inactive = np.zeros(spine_activity.shape) * np.nan
dend_active = np.zeros(dendrite_activity.shape) * np.nan
dend_inactive = np.zeros(dendrite_activity.shape) * np.nan

for i in range(spine_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(spine_dFoF[:, i], spine_activity[:, i])
    d_active, d_inactive = get_active_inactive(dendrite_dFoF[:, i], dendrite_activity[:, i])
    spine_active[:, i] = s_active
    spine_inactive[:, i] = s_inactive
    dend_active[:, i] = d_active
    dend_inactive[:, i] = d_inactive

In [ ]:
# Explore the traces
%matplotlib qt

for i in range(spine_activity.shape[1]):
    plt.figure()
    plt.plot(spine_inactive[:, i], color="black")
    plt.plot(spine_active[:, i], color="blue")
    plt.ylim(bottom=-0.5, top=1.5)

In [11]:
# Group plot
%matplotlib qt
save_path = r"C:\Users\Jake\Desktop\Figures\Figure_drafts"

x = np.arange(len(spine_active)) / 60
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
title = "JW202_FOV3_Early_All_spine_traces"
fig.suptitle(title)
ax["A"].plot(x, dend_inactive[:, 0], color="red", alpha=0.3, linewidth=.8)
ax["A"].plot(x, dend_active[:, 0], color="red", linewidth=.8)

for  i in reversed(range(spine_activity.shape[1])):
#for i in reversed(range(30,37)):
    if np.isnan(spine_active[:, i]).all():
        continue
    print(i)
    ax["A"].plot(x[:], spine_inactive[:, i] + i +1.1, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(x[:], spine_active[:, i] + i + 1.1, color="blue", linewidth=.8)

fname = os.path.join(save_path, title)
fname = os.path.join(save_path, "Example_coincidental_activity_traces")
#fig.savefig(fname + ".pdf")

38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [ ]:
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,10))
x = np.arange(len(spine_active)) / 60
ax["A"].plot(x[5100:6300], dendrite_activity[5100:6300, 0], color="red", alpha=1, linewidth=.8)
for i in reversed(range(30,37)):
    ax["A"].plot(x[5100:6300], spine_activity[5100:6300, i] + i +1.1, color="black", alpha=1, linewidth=.8)
    
fname = os.path.join(save_path, "Example_coincidental_activity_binary")
fig.savefig(fname + ".pdf")

In [ ]:
import random
import numpy as np
a = np.array([1,2,3,4,5,6,7,8,9,10])

b = random.sample(list(a), 5)
print(b)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data = np.random.random(size=(1000,22))
fig, axs = plt.subplots(2,3, figsize=(7,4)
)
gridspec = axs[0, 0].get_subplotspec().get_gridspec()
# clear the left column for the subfigure:
for a in axs[:, 0]:
    a.remove()
for a in axs[:, 1]:
    a.remove()
# make the subfigure in the empty gridspec slots:
subfig = fig.add_subfigure(gridspec[:, 0])
axsLeft = subfig.subplots(3,1,sharex=True,                                         )                       
               interpolation='nearest')
a2 = axsLeft[1].imshow(data[:,5:10].T, cmap="hot", 
               vmin=0.0, vmax=1.0, aspect='auto',  
               interpolation='nearest')
a3 = axsLeft[2].imshow(data[:,11:].T, cmap="hot", 
               vmin=0.0, vmax=1.0, aspect='auto',  
               interpolation='nearest')
subfig.colorbar(a1, shrink=0.3, ax=axsLeft, location='right', orientation="vertical")
axs[0,2].plot(data[:, 0])
subfig2 = fig.add_subfigure(gridspec[:, 1])
axsLeft2 = subfig2.subplots(1,1,sharex=True)
a12 = axsLeft2.imshow(data[:,:5].T, cmap="hot", 
               vmin=0.0, vmax=1.0, aspect='auto',  
               interpolation='nearest')


In [ ]:
a = ["JW111", "JW111", "JW222", "JW222"]
b = ["FOV1", "FOV2","FOV1", "FOV2"]
c = np.array([(x=="JW111") * (y=="FOV2") for x,y in zip(a,b)]).astype(bool)

In [ ]:
c

In [ ]:
import numpy as np
a = np.zeros((5,5))

In [ ]:
a = np.array([1,2,3,4,5])
b = np.ones(a.shape)

In [ ]:
a = np.append(a, np.array([np.nan, np.nan]))

In [ ]:
np.percentile(a, 50)

In [ ]:
def exp_func(t, A, K, C):
    return A * np.exp(-K * t) + C

In [ ]:
plt.figure()
plt.plot(np.arange(100), exp_func(np.arange(100), 100, 0.005, 6))

In [ ]:
fitted = exp_func(np.arange(100), 100, 0.000005, 6)
m = (fitted[-1] - fitted[0]) / (
                    np.arange(100)[-1] / 60
                    - np.arange(100)[0] / 60
                )
m